In [1]:
import pandas as pd
import os
import re
from tqdm import notebook, trange, tqdm
import spacy
import ast
import numpy as np
from fuzzywuzzy import fuzz
import itertools
import logging
from seqeval.metrics import f1_score, precision_score, recall_score
import argparse
import glob
import random
from sklearn.model_selection import KFold
import pickle
spacy.require_gpu()


True

## Loading in Dataset

In [7]:
df = pd.read_csv('../clean_data.csv')
notebook.tqdm.pandas()

D:\Anaconda\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Spacy

In [ ]:
#!python -m spacy download nl_core_news_sm

#nlp = spacy.load('../spacy1')
#nlp = spacy.load('nl_core_news_sm')
#nlp = spacy.load('../spacytest')
#nlp = spacy.load('../d=0.8_distance=80')
nlp = spacy.load('../Spacy models/kfold_3', disable=['parser', 'tagger', 'textcat'])

In [ ]:
for ent in nlp(df.text[15]).ents:
    if ent.label_ == 'PERSON':
        print(ent.text, ent.label_)

In [2]:
def compare(ner, true, distance):
    tp = 0
    fp = 0
    fn = 0
    false_negatives = []
    false_positives = []
    corrector = []
    corrector2 = []
    for x in true:
        # Check Normal Names
        if x in ner:
            tp += 1
            corrector.append(x)
            corrector2.append(x)
            
        else:
            # Check Removed Last Names 
            for y in ner:
                if type(y) == tuple:
                    if y[0] == x:
                        tp += 1
                        corrector.append(y[0])
                        corrector2.append(y)

                    elif y[1] == x:
                        tp += 1
                        corrector.append(y[1])
                        corrector2.append(y)
                        
                    else:
                        if fuzz.ratio(x.lower(),y[0].lower()) >= distance:
                            tp += 1
                            corrector.append(x)
                            corrector2.append(y)
                            
                        elif fuzz.ratio(x[1].lower(),y.lower()) >= distance:
                            tp += 1
                            corrector.append(x)
                            corrector2.append(y)

                elif fuzz.ratio(x.lower(),y.lower()) >= distance:
                    tp += 1
                    corrector.append(x)
                    corrector2.append(y)

        if x not in corrector:
            fn += 1
            false_negatives.append([x, ner])

    for y in ner:
        if y not in corrector2:
            fp += 1
            false_positives.append([y, true])
    #fp = len(ner) - tp
    return tp, fp, fn, false_negatives, false_positives
    

In [3]:
def spacy_name_getter(text, nlp, distance):
    doc = nlp(text)
    holder = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON' and ' ' in ent.text:
            if ent.text not in holder:
                check = True
                for x in holder:
                    if fuzz.ratio(x, ent.text) >= distance:
                        check = False
                        break
                if check == True:
                    holder.append(ent.text)
    return holder
    

In [4]:
def bertje_test(ner_clean):
    holder = []
    prev = None
    for x in ast.literal_eval(ner_clean):
        if x[-1].strip() == 'B-PER':
            holder.append(x[0])
            prev = x[-1].strip()
        elif x[-1].strip() == 'I-PER':
            if prev == 'B-PER' or prev == 'I-PER':
                holder[-1] += ' ' + x[0]
                prev = x[-1].strip()
            else:
                holder.append(x[0])
                prev = x[-1].strip()
        else:
            prev = x[-1].strip()
        
    return holder

In [5]:
def true_name_getter(true):
    holder = []
    for x in ast.literal_eval(true):
        if x['tussenvoegsel'] != None:
            holder.append(x['voornaam'] + " " + x['tussenvoegsel'] + " " + x['achternaam'])
        elif x['voornaam'] and x['achternaam'] != None:
            holder.append(x['voornaam'] + " " + x['achternaam'])
    return holder

In [6]:
def NER_test(row, model, levenshtein, nlp):
    if model == 'SpaCy':
        ner = spacy_name_getter(row.text, nlp, levenshtein)
    if model == 'BERTje':
        ner = bertje_test(row.ner_clean)
    true = true_name_getter(row.namen)
    tp, fp, fn, false_negatives, false_positives = compare(ner, true, levenshtein)
    #return {'tp' : tp, 'fp': fp, 'fn': fn}
    return {'tp' : tp, 'fp': fp, 'fn': fn, 'false_negatives': false_negatives, 'false_positives':false_positives}

In [ ]:
ner_result = df.progress_apply(NER_test, args=('SpaCy', 90, nlp), axis=1)

In [ ]:
total_fp = sum([x['fp'] for x in ner_result])
total_tp = sum([x['tp'] for x in ner_result])
total_fn = sum([x['fn'] for x in ner_result])

In [ ]:
total_tp, total_fp, total_fn, total_tp / (total_tp + total_fn), total_tp / (total_tp + total_fp)

## Spacy Tests

### Oud SpaCy model
#### Base SpaCy
TP: 23862  
FP: 190907  
FN: 45972  
Recall: 0.34169602199501675  
Precision: 0.11110542024221373  

#### SpaCy met achternaam removal
TP: 26937  
FP: 187832  
FN: 45074  
Recall: 0.3740678507450251  
Precision: 0.12542312903631345  

#### SpaCy met Levenshtein distance 2 (fuzzy ratio 90) en achternaam removal
TP: 41931  
FP: 173001  
FN: 34235  
Recall: 0.5505212299451199  
Precision: 0.1950896097370331  

#### SpaCy met Levenshtein distance 3 (fuzzy ratio 85) en achternaam removal
TP: 47121  
FP: 167997  
FN: 30955  
Recall: 0.6035273323428454  
Precision: 0.2190472205952082  

#### SpaCy trained model met fuzzy ratio 90
TP: 20186  
FP: 14602  
FN: 45247  
Recall: 0.3084987697339263  
Precision: 0.5802575600781879  

#### SpaCy trained model met fuzzy ratio 80
TP: 22992  
FP: 11796  
FN: 42441  
Recall: 0.3513823300169639  
Precision: 0.6609175577785443  

### Nieuw SpaCy model
#### Base SpaCy

#### SpaCy met Levenshtein distance 2 (fuzzy ratio 90) en achternaam removal
TP: 27190  
FP: 241577  
FN: 38243  
Recall: 0.4155395595494628  
Precision: 0.10116569370495634

#### SpaCy met Levenshtein distance 2 (fuzzy ratio 80) en achternaam removal
TP: 27190  
FP: 241577  
FN: 38243  
Recall: 0.5240322161600416  
Precision: 0.12757890663660346

#### SpaCy ner trained model met fuzzy ratio 80
TP: 28833  
FP: 17218  
FN: 36600  
Recall: 0.44064921369950943  
Precision: 0.6261101821893118

#### SpaCy notaris annotatie fuzzy ratio 90 train 0.1 model met fuzzy ratio 80
TP: 35233  
FP: 20138  
FN: 30200  
Recall: 0.538459187260251  
Precision: 0.6363078145599682

#### SpaCy notaris annotatie fuzzy ratio 80 train 0.1 model met fuzzy ratio 80, dropout rate 0.5
TP: 43444  
FP: 27132  
FN: 21989  
Recall: 0.6639463267770085  
Precision: 0.6155633643164815

#### SpaCy notaris annotatie fuzzy ratio 80 train 0.1 model met fuzzy ratio 80, dropout rate 0.2
TP: 43004  
FP: 25968  
FN: 22429  
Recall: 0.6572218910947075  
Precision: 0.6234993910572406

#### SpaCy notaris annotatie fuzzy ratio 80 train 0.1 model met fuzzy ratio 80, dropout rate 0.8
TP: 48857  
FP: 38664  
FN: 16576  
Recall: 0.7466721684776795  
Precision: 0.5582317386684339

#### SpaCy notaris annotatie fuzzy ratio 80 train 0.5 model met fuzzy ratio 80, dropout rate 0.5
TP: 48857  
FP: 38664  
FN: 16576  
Recall: 0.7894850647144225  
Precision: 0.5719592704486964

#### SpaCy notaris annotatie fuzzy ratio 80 train 0.5 model met fuzzy ratio 90, dropout rate 0.5
TP: 23513  
FP: 21172  
FN: 8860  
Recall: 0.7263151391591759  
Precision: 0.5261944724180374

#### SpaCy notaris annotatie fuzzy ratio 80 train 0.5 model met fuzzy ratio 80, dropout rate 0.8
TP: 51102  
FP: 35873  
FN: 14331  
Recall: 0.7809820732657833  
Precision: 0.587548146018971

## Training Spacy

#### Creating Training Data

In [ ]:
raw_train = df.sample(frac=0.01, random_state=0)
nlp = spacy.load('nl_core_news_sm')

In [3]:
def train_data_creator_raw(row, distance):
    doc = nlp(row.text)
    true = true_name_getter(row.namen)
    locs = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON' and ' ' in ent.text:
            #holder.append(ent.text)
            check = False
            corrector = []
            if ent.text in true:
                check = True
            else:
                for x in true:
                    if check == False:
                        if fuzz.ratio(x, ent.text) >= distance:
                            check = True
            if check == True:
                holder = []
                for x in ent:
                    holder.append(x.i)
                locs.append((doc[holder[0]].idx, doc[holder[-1]].idx + len(doc[holder[-1]].text), 'PERSON'))
                    
           
    
    
    return (row.text, {'entities': locs})

def match_neighbour(start, end, true, prev, distance):
    if true == []:
        return (start, end, "PERSON"), prev.i
    if prev.i == len(prev.doc) - 1:
        return (start, end, "PERSON"), prev.i
    if fuzz.ratio(true[0], prev.nbor().text) >= distance:
        return match_neighbour(start, prev.nbor().idx + len(prev.nbor()), true[1:], prev.nbor(), distance)
    else:
        return (start, end, "PERSON"), prev.i

def train_data_creator(row, distance):
    true = true_name_getter(row.namen)
    doc = nlp(row.text)
    locs = []
    prev = 0
    for token in doc:
            for x in true:
                if token.i > prev:
                    split = x.split(' ')
                    if fuzz.ratio(split[0], token.text) >= distance:
                        result, prev = match_neighbour(token.idx, token.idx + len(token), split[1:], token, distance)
                        if result not in locs:
                            locs.append(result)
    return (row.text, {'entities': locs})





In [4]:
#!/usr/bin/env python
# coding: utf8
"""Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.0.0+
Last tested with: v2.1.0
"""
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding





# @plac.annotations(
#     model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#     output_dir=("Optional output directory", "option", "o", Path),
#     n_iter=("Number of training iterations", "option", "n", int),
# )
def main(train=None, model=None, output_dir=None, n_iter=100, dropout=0.5, start=4.0, end=32.0, compound=1.001, test=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    TRAIN_DATA = train
    if model is not None:
        nlp = spacy.load(model, disable=['parser', 'tagger', 'textcat'])  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in notebook.tqdm(range(n_iter)):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(start, end, compound))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=dropout,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            #print("Losses", losses)

    # test the trained model
    #for text, _ in TRAIN_DATA:
        #doc = nlp(text)
        #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        if type(test) != type(None):
            print("Loading from", output_dir)
            nlp2 = spacy.load(output_dir, disable=['parser', 'tagger', 'textcat'])
            ner_result = test.progress_apply(NER_test, args=('SpaCy', 90, nlp2), axis=1)
            total_fp = sum([x['fp'] for x in ner_result])
            total_tp = sum([x['tp'] for x in ner_result])
            total_fn = sum([x['fn'] for x in ner_result])
            print('Recall: ' + str(total_tp / (total_tp + total_fn)))
            print('Precision: ' + str(total_tp / (total_tp + total_fp)))

In [ ]:
nlp = spacy.load('nl_core_news_sm', disable=['parser', 'tagger', 'textcat'])
train = [train_data_creator(x, 80) for x in notebook.tqdm(df.itertuples(),  total=df.shape[0])]


In [ ]:
type(None) != type(None)

In [ ]:
df['traindata'] = train
c= 0
for train_index, test_index in kf.split(df):
    c += 1
    df.loc[train_index].to_json('train_kfold_' + str(c) + '.json')
    df.loc[test_index].to_json('test_kfold' + str(c)+ '.json')

In [ ]:
train = pd.read_json('train_kfold_1.json').traindata.tolist()
test = pd.read_json('test_kfold1.json')
#main(train=train, model='nl_core_news_sm', output_dir='../kfold_' + '1', dropout=0.5, test=test)
if type(test) != type(None):
            print("Loading from", '../kfold_' + '1')
            nlp2 = spacy.load('../kfold_' + '1', disable=['parser', 'tagger', 'textcat'])
            ner_result = test.progress_apply(NER_test, args=('SpaCy', 90, nlp2), axis=1)
            total_fp = sum([x['fp'] for x in ner_result])
            total_tp = sum([x['tp'] for x in ner_result])
            total_fn = sum([x['fn'] for x in ner_result])
            print('Recall: ' + str(total_tp / (total_tp + total_fn)))
            print('Precision: ' + str(total_tp / (total_tp + total_fp)))


In [ ]:
train = pd.read_json('train_kfold_2.json').traindata.tolist()
test = pd.read_json('test_kfold2.json')
#main(train=train, model='nl_core_news_sm', output_dir='../kfold_' + '2', dropout=0.5, test=test)
if type(test) != type(None):
            print("Loading from", '../kfold_' + '2')
            nlp2 = spacy.load('../kfold_' + '2', disable=['parser', 'tagger', 'textcat'])
            ner_result = test.progress_apply(NER_test, args=('SpaCy', 90, nlp2), axis=1)
            total_fp = sum([x['fp'] for x in ner_result])
            total_tp = sum([x['tp'] for x in ner_result])
            total_fn = sum([x['fn'] for x in ner_result])
            print('Recall: ' + str(total_tp / (total_tp + total_fn)))
            print('Precision: ' + str(total_tp / (total_tp + total_fp)))


In [ ]:
train = pd.read_json('train_kfold_3.json').traindata.tolist()
test = pd.read_json('test_kfold3.json')
main(train=train, model='nl_core_news_sm', output_dir='../kfold_' + '3', dropout=0.5, test=test)

In [ ]:
train = pd.read_json('train_kfold_4.json').traindata.tolist()
test = pd.read_json('test_kfold4.json')
main(train=train, model='nl_core_news_sm', output_dir='../kfold_' + '4', dropout=0.5, test=test)

In [ ]:
train = pd.read_json('train_kfold_5.json').traindata.tolist()
test = pd.read_json('test_kfold5.json')
main(train=train, model='nl_core_news_sm', output_dir='../kfold_' + '5', dropout=0.5, test=test)

## Folds

#### 1
Recall: 0.694800301431801  
Precision: 0.5690655474632761

#### 2
Recall: 0.7323612417685795  
Precision: 0.5351741521539871

#### 3
Recall: 0.7527012631258561  
Precision: 0.5349340255245512

#### 4
Recall: 0.7279757391963608  
Precision: 0.5448252383113935

#### 5
Recall: 0.7403654230001535  
Precision: 0.5438141423254765

In [ ]:
main(train=, model='nl_core_news_sm', output_dir='../d=0.8_distance=80_t=0.5', dropout=0.8, test=)

## Structured data test

In [ ]:
def testje(row, distance):
    true = true_name_getter(row.namen)
    doc = nlp(row.text)
    locs = []
    for x in true:
        for token in doc:
            split = x.split(' ')
            if fuzz.ratio(split[0], token.text) >= distance:
                result, prev = match_neighbour(token.idx, token.idx + len(token), split[1:], token, distance)
                entity = row.text[result[0]:result[1]]
                if fuzz.partial_ratio(entity, x) >= 90 and x not in locs:
                    locs.append(entity)
    return (true, locs)


## Fouten Analyse

In [8]:
nlp = spacy.load('../Spacy models/kfold_3', disable=['parser', 'tagger', 'textcat'])

In [9]:
test = pd.read_json('kfolds/test_kfold3.json')
if type(test) != type(None):
            ner_result = test.progress_apply(NER_test, args=('SpaCy', 90, nlp), axis=1)
            total_fp = sum([x['fp'] for x in ner_result])
            total_tp = sum([x['tp'] for x in ner_result])
            total_fn = sum([x['fn'] for x in ner_result])
            false_negatives = [x['false_negatives'] for x in ner_result]
            false_positives = [x['false_positives'] for x in ner_result]
            print('Recall: ' + str(total_tp / (total_tp + total_fn)))
            print('Precision: ' + str(total_tp / (total_tp + total_fp)))


Recall: 0.7559266438049799
Precision: 0.7305475504322767


In [10]:
# recall = 0.7628775718663359
# precision = 0.657114127702363


recall = 0.7559266438049799
precision = 0.7305475504322767

In [11]:
print('F1: ' + str(2*((precision*recall) / (precision + recall))))

F1: 0.7430204440536382


In [12]:
len(false_negatives)

1307

In [13]:
len(false_positives)

1307

In [14]:
total_fp, total_tp, total_fn

(1870, 5070, 1637)

### Mogelijke fouten

#### False Positives
- Not an actual person tagged as person
- Name was tagged but too dissimilar from annotated name due to HTR
- The name was only partially tagged
- Name was not annotated

#### False Negatives
- Name was tagged but too dissimilar from annotated name due to HTR
- The name was only partially tagged
- Name was not tagged

In [31]:
# Last: 344
# Disimilar Negatives: 127
# Not Tagged Negatives: 313
# Partial Negatives: 60
    
last = 344
disimilar = 127
not_tagged = 313
partial = 60
with open("disimilar_index.txt", "rb") as fp:   # Unpickling
    disimilar_index = pickle.load(fp)
with open("partial_index.txt", "rb") as fp:   # Unpickling
    partial_index = pickle.load(fp)
with open("not_tagged_index.txt", "rb") as fp:   # Unpickling
    not_tagged_index = pickle.load(fp)

In [28]:
for x in range(len(false_negatives[last:])):
    for y in false_negatives[x + last]:
        print('{:30} | {}'.format("Annotated Person: " + str(x + last), 'Tagged Persons'))
        print('-' * 108)
        print('{:30} | {}'.format(y[0], y[1]))
        check = False
        while check != True:
            answer = input()
            if answer == 'dis':
                disimilar += 1
                disimilar_index.append((x, y[0]))
                check = True
            elif answer == 'nt':
                not_tagged += 1
                not_tagged_index.append((x, y[0]))
                check = True
            elif answer == 'par':
                partial += 1
                partial_index.append((x, y[0]))
                check = True
            elif answer == 'text':
                print(test.iloc[x + last].text)
            elif answer == 'stop':
                last += x
                break
            else:
                print("Invalid input please enter 'dis' or 'nt'.")
        if answer == 'stop':
            break
    if answer == 'stop':
        break

print('Last: ' + str(last))
print('Not Tagged Negatives: ' + str(not_tagged))
print('Disimilar Negatives: ' + str(disimilar))
print('Partial Negatives: ' + str(partial))
with open("disimilar_index.txt", "wb") as fp:   #Pickling
    pickle.dump(disimilar_index, fp)
with open("partial_index.txt", "wb") as fp:   #Pickling
    pickle.dump(partial_index, fp)
with open("not_tagged_index.txt", "wb") as fp:   #Pickling
    pickle.dump(not_tagged_index, fp)

    

Annotated Person: 329          | Tagged Persons
------------------------------------------------------------------------------------------------------------
Sjoerd Jacobsz                 | ['raadman Jacobsz', 'george Lesbi', 'Jan Mor']
nt
Annotated Person: 329          | Tagged Persons
------------------------------------------------------------------------------------------------------------
George Leslie                  | ['raadman Jacobsz', 'george Lesbi', 'Jan Mor']
dis
Annotated Person: 329          | Tagged Persons
------------------------------------------------------------------------------------------------------------
Hans Horneman                  | ['raadman Jacobsz', 'george Lesbi', 'Jan Mor']
nt
Annotated Person: 329          | Tagged Persons
------------------------------------------------------------------------------------------------------------
Jan Mon                        | ['raadman Jacobsz', 'george Lesbi', 'Jan Mor']
dis
Annotated Person: 330          | Tagge

In [19]:
# Last: 422
# Not Person Positives: 92
# Not Tagged Positves: 97
# Disimilar Positives: 211
# Partial Positives: 136


last = 422
not_person = 92
not_annotated = 97
dis = 211
par = 136
with open("dis_index.txt", "rb") as fp:   # Unpickling
    dis_index = pickle.load(fp)
with open("par_index.txt", "rb") as fp:   # Unpickling
    par_index = pickle.load(fp)
with open("np_index.txt", "rb") as fp:   # Unpickling
    np_index = pickle.load(fp)
with open("na_index.txt", "rb") as fp:   # Unpickling
    na_index = pickle.load(fp)


In [20]:
(not_person + not_annotated + dis + par)

536

In [18]:
c= 0
p = 0
for x in range(len(false_positives[last:])):
    c += 1
    
    for y in false_positives[x + last]:
        print('{:30} | {}'.format("Tagged Person: " + str(x + last), 'Annotated Persons'))
        print('-' * 108)
        print('{:30} | {}'.format(y[0], y[1]))
        check = False
        while check != True:
            answer = input()
            if answer == 'np':
                not_person += 1
                np_index.append((x + last, y[0]))
                check = True
            elif answer == 'na':
                not_annotated += 1
                na_index.append((x + last, y[0]))
                check = True
            elif answer == 'dis':
                dis += 1
                dis_index.append((x + last, y[0]))
                check = True
            elif answer == 'par':
                par += 1
                par_index.append((x + last, y[0]))
                check = True
            elif answer == 'text':
                print(test.iloc[x + last].text)
            elif answer == 'stop':
                last += x
                break
            else:
                print("Invalid input please enter 'dis' or 'nt'.")
        if answer == 'stop':
            break
    if answer == 'stop':
        break
    elif c - p == 20:
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('Total cases evaluated: ' + str(not_person + not_annotated + dis + par))
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        p = c
print('Last: ' + str(last))
print('Not Person Positives: ' + str(not_person))
print('Not Tagged Positves: ' + str(not_annotated))
print('Disimilar Positives: ' + str(dis))
print('Partial Positives: ' + str(par))
with open("dis_index.txt", "wb") as fp:   #Pickling
    pickle.dump(dis_index, fp)
with open("par_index.txt", "wb") as fp:   #Pickling
    pickle.dump(par_index, fp)
with open("np_index.txt", "wb") as fp:   #Pickling
    pickle.dump(np_index, fp)
with open("na_index.txt", "wb") as fp:   #Pickling
    pickle.dump(na_index, fp)


Tagged Person: 157             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Coert Saaed                    | ['Josua van der Poorten', 'Daniel Deutz']
text
470 No: 470 Janarej JranuFfarth 1eg phembrs 1755 Joofkom dieff hegt voorf hijt Jasau Er gnenie Snsone un Janne alas Pefsnlriek nue didn opde Janne Pomas Copt der Sama oond Can haupeues Gelhud Jolk Coert Saaed Zuifft nursaugen Gestiene eaij Prfauirg Shut adris ina Sen Crijheu Gagtns Nisiung Borven Sanil Pietess ru Sijne S Geij Amsterdam Vor mij aejan In ondes Jans Jolua van der Poorten en gorose ud resring frff �30Cbre 1755 Hanas Cns Op Heden den 8 de soberties Jaar en diselendeleven hendert vijffen Vurig belik Jan herleid Notaris Publicq bij den Ede: Hove van Holland Geadmitteert binnen Amsterdamt Resideerende Jan Verke ken van dieg teer Todaa van der Poerten kegpman binnen diese Stad Geenterpisteert de Heeren Daniel deusz enhonenMede kooplieden alhi

txt
Invalid input please enter 'dis' or 'nt'.
text
N: 535 Verklaring Gpaaserd en 11 November 1745 a al:15:fMacklaudOp Heden Den 1 November d Jaars 1756 Comparde voor mij Jan herleid ris publicq bij den Ed: Hove van o oblnd geadmittteerd binnen Amsterdam resider d5 ari535 Compareerden den 11e: nov: 1745 d' E E Pieter Jacobsz Stuurman Christiaan Andriesz bootsman Swert hanssen Timmerman en Christiaan bnosse Matroos alle vier van genoegen ouderdom en in die bedieningen nu Jonget gevaren hebbende op 't Schip genaemd de Post galeij daar Schipper op is foost Jacobsen ter wens acquisitie Sij Comparanten hebben getuijgt en verslaart Dat hun voorne: Schip in de maand Sulij deses Jaers 1745 te genua en te ports Marice is beladen met vaten olij, Aluijn, Kassen Zeep vier Marmore blocken en eenige wijnige Stukgoederen meer, alle welke goederen & koopmanschappen in hun Schip behoorlijk gestuwt en geplaast geworden en de vaten onderslagen en vast gelegt geworden Sijn Dat het gedagte Schip voor en op 

par
Tagged Person: 167             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Albent Htensbing               | ['Andries Boede', 'Coenraad Harmen Christiaan', '... Wiel', '... Kemton', 'Hendrik Reppen']
np
Tagged Person: 168             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Cornelis Jansz                 | ['Leendert Pietersz Snobeger', 'Adriaan Karreman', 'Pieter Leendertsz Snobeger', 'Dirk Adriaansz de Boer', 'Cornelis Jansz Kort']
par
Tagged Person: 169             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Sijbrand Vornis                | ['Rijk Lover', 'Adriana van der Toren', 'Arnoldus van den Ende', 'Sybrand Vorms']
dis
Tagged Person: 170             | Annotated Persons
---------------------------------------------------

par
Tagged Person: 177             | Annotated Persons
------------------------------------------------------------------------------------------------------------
frans Werree                   | ['Frans Overzee', 'Pieter Leur', 'Fredrik Dammers', 'Samuel Hoedemaker', 'Pieter Cornelisz Louwe', 'Harmen Swavelstok']
dis
Tagged Person: 179             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Condigonde Siems               | ['Claas Non', 'Hendrik Boele', 'Cordigonda Siems', 'Antoni Croezen', 'Catharina Siems', 'Volkert Lubke']
dis
Tagged Person: 179             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Claas Non de Jonge             | ['Claas Non', 'Hendrik Boele', 'Cordigonda Siems', 'Antoni Croezen', 'Catharina Siems', 'Volkert Lubke']
text
Dit Testament van Man & vrouw vervat geene makingen van fideicommis en heb

par
Tagged Person: 181             | Annotated Persons
------------------------------------------------------------------------------------------------------------
geertruij van der              | ['Geertruij van der Ven', 'Jan Hendrik Ernst', 'Andries Andriesz', 'Barend van der Does', 'Adam Andriesz', 'Burgert Hendriksz', 'Gesijna van der Does']
par
Tagged Person: 181             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Jan hendrick                   | ['Geertruij van der Ven', 'Jan Hendrik Ernst', 'Andries Andriesz', 'Barend van der Does', 'Adam Andriesz', 'Burgert Hendriksz', 'Gesijna van der Does']
par
Tagged Person: 182             | Annotated Persons
------------------------------------------------------------------------------------------------------------
gerrit Veldman                 | ['Jan Klinkert', 'Isaak van der Ven', 'Pieter Christaansz', 'Jacob Jansen', 'Maria Tadsen']
text
No: 222 

...

na
Tagged Person: 378             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Docke Auberts                  | ['Tjipke Bernardus', 'Focke Huberts', 'Hendrik Jansz', 'Dirk Michiels']
dis
Tagged Person: 378             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Dirk Micl                      | ['Tjipke Bernardus', 'Focke Huberts', 'Hendrik Jansz', 'Dirk Michiels']
dis
Tagged Person: 378             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Jan Tipke Berhardus            | ['Tjipke Bernardus', 'Focke Huberts', 'Hendrik Jansz', 'Dirk Michiels']
dis
Tagged Person: 379             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Dierk Pierks                  

dis
Tagged Person: 383             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Jan hennicus                   | ['Elsina Masman', 'Wessel Scholten', 'Jan Henricus Le Blon', 'Evert Laurens Cramer', 'Gesina Jacobs Mulder', 'Pieter Rennink']
par
Tagged Person: 383             | Annotated Persons
------------------------------------------------------------------------------------------------------------
gesina Jacobmuldz              | ['Elsina Masman', 'Wessel Scholten', 'Jan Henricus Le Blon', 'Evert Laurens Cramer', 'Gesina Jacobs Mulder', 'Pieter Rennink']
dis
Tagged Person: 385             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Hendrik Sellen                 | ['Cornelis Cornelisz', 'Hendrik Jelles', 'Rienk Hielkes', 'Wouter Eelkes']
dis
Tagged Person: 385             | Annotated Persons
-------------------------

dis
Tagged Person: 398             | Annotated Persons
------------------------------------------------------------------------------------------------------------
foppe Pietens                  | ['Claas Gauwes', 'Foppe Pietersz', 'Fijke Pietersz', 'Jan Simensz']
dis
Tagged Person: 398             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Ippe Pietersz                  | ['Claas Gauwes', 'Foppe Pietersz', 'Fijke Pietersz', 'Jan Simensz']
dis
Tagged Person: 398             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Jan Semons                     | ['Claas Gauwes', 'Foppe Pietersz', 'Fijke Pietersz', 'Jan Simensz']
dis
Tagged Person: 399             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Melchert van                   | ['... Re

na
Tagged Person: 409             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Jan de Jong                    | ['Hendrik de Wind', 'Jan de Jong Hendrikszoon']
par
Tagged Person: 409             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Hermanus Coops fledderus       | ['Hendrik de Wind', 'Jan de Jong Hendrikszoon']
np
Tagged Person: 410             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Sara Temmina                   | ['Alewijn Crombosch', 'Rebecca van der Loveren']
text
Calkoen Dit Testament van man & vrouw vervat geen fideicommis & No: 277 hebbende testateur verklaerd te Samen niet boven de � 50000:- gegoed te Sijn Testament gecasseerd den 10e: Junij 1749Nf5 Op heden den teniee des Jaars Een drijvend boven hondert nigen en ve

na
Tagged Person: 413             | Annotated Persons
------------------------------------------------------------------------------------------------------------
Gerrit Jacob                   | ['Barend Linlan', 'Jan Joosten', 'Ariaan Doren', 'Adam Beer', 'Stoffel Scholsx', 'Jan Hendriksz', 'Gerrit Prins']
text
No: 294 Verklaring Gepassert Den 19 JunjongCalkoenOp heden den 190 Janij des Jaars 1755 Compareerden voor mij Jan verleij Notaris publiek bij den Edhove van Hollander geadmiteerd binnen Amsterdam Resideerene d' E E: Barant19e: fnijl g1 d' E E. Barend Linlau Stuurman Jan Joosten timmerman Ariaan Doren kok Adam Beer Stoffel Schols & Jan hHendriksz matrosen alle Van genoegen ouderdenden in die bedieningen nu Jongst gevaaren hebbende op 't Schip de Jonge daar Schipper op is Gerrit Jacob terwiens requisitie Prins Sij Comparanten hebben getuijgt en verklaert Dat hun voornoemde Schip in de maanden februarij & Maert deses Jaers 1749 te Livorno is beladen met Stukgoederen van diverse S